In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import pygraphviz
from pandas import read_csv
import seaborn as sns
import lightgbm as lgb

from sklearn.datasets import make_classification
from sklearn.dummy import DummyClassifier

from lightgbm import LGBMClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
import os

# Summary
Methods used

Non Deep-Learning:
- LightGBM
- HistGradientBoostingClassifier
- XGBoost

Deep-Learning: (in progress)
- FeedForward/MLP
    - single works
    - all species is giving back weird error (check dataset first prolly there)
- Multi-task Learning (in progress)

# Non DL

## Data

### Preprocessed

#### LOAD Single or All

@Josefa:

I checked the database again and find out how they calculated SD1,SD2,SD3 and SDT 

Basically for each genome, the HGTdb website gave its mean value and standard deviation for GC1,GC2,GC3 and GCT 
eg (https://usuaris.tinet.cat/debb/HGT/ecoli.d/welcome.html)

basically SD1 = (GC1 of gene - GC1 mean)/(standard deviation)

basically we can actually delete, for every gene GC1,GC2,GC3 and GCT and uses for each gene
SD1,SD2,SD3 and SDT 

My though process on data type D:
To create a model that does not look at the GC content of genome under study.. instead how far off genes (in comparision to host genome)

Added another data type E:
basically just data type D with strand, AADev, length, Mah (Data type B without GC)... Data B currently is the best performing

In [1]:
from utils.machine_learning.data_loader import ml_load_species

In [3]:
X_ecoli_A, y_ecoli_A = ml_load_species('ecoli', 'A')
X_ecoli_B, y_ecoli_B = ml_load_species('ecoli', 'B')
X_ecoli_C, y_ecoli_C = ml_load_species('ecoli', 'C')
X_ecoli_D, y_ecoli_D = ml_load_species('ecoli', 'D')
X_ecoli_E, y_ecoli_E = ml_load_species('ecoli', 'E')
print(X_ecoli_A.shape)
print(X_ecoli_B.shape)
print(X_ecoli_C.shape)
print(X_ecoli_D.shape)
print(X_ecoli_E.shape)


(4276, 13)
(4276, 12)
(4276, 10)
(4276, 4)
(4276, 8)


In [4]:
X_all, y_all = ml_load_species('all','E')
print(X_all.shape)

(195187, 8)


#### Partition

In [5]:
# A
X_ecoli_A_train, X_ecoli_A_test, Y_ecoli_A_train, Y_ecoli_A_test = train_test_split(X_ecoli_A, y_ecoli_A, test_size=0.20, shuffle=False)
# B wocogcode
X_ecoli_B_train, X_ecoli_B_test, Y_ecoli_B_train, Y_ecoli_B_test = train_test_split(X_ecoli_B, y_ecoli_B, test_size=0.20, shuffle=False)
# C wocatfeat
X_ecoli_C_train, X_ecoli_C_test, Y_ecoli_C_train, Y_ecoli_C_test = train_test_split(X_ecoli_C, y_ecoli_C, test_size=0.20, shuffle=False)
# D 
X_ecoli_D_train, X_ecoli_D_test, Y_ecoli_D_train, Y_ecoli_D_test = train_test_split(X_ecoli_D, y_ecoli_D, test_size=0.20, shuffle=False)
# E 
X_ecoli_E_train, X_ecoli_E_test, Y_ecoli_E_train, Y_ecoli_E_test = train_test_split(X_ecoli_E, y_ecoli_E, test_size=0.20, shuffle=False)

# print(X_coliEC_D_train.shape, Y_coliEC_D_train.shape)

Shuffle=False make sure that we can compare the effect on features

In [6]:
print(X_ecoli_A_train[0])
print(X_ecoli_B_train[0])
print(X_ecoli_C_train[0])
print(X_ecoli_D_train[0])
print(X_ecoli_E_train[0])

[0 63 nan 22.7 -5.9 68.2 6.3 63.6 1.0 51.5 0.0 422.6 1]
[  0.   63.   22.7  -5.9  68.2   6.3  63.6   1.   51.5   0.  422.6   1. ]
[ 63.   22.7  -5.9  68.2   6.3  63.6   1.   51.5   0.  422.6]
[-5.9  6.3  1.   0. ]
[  0.   63.   -5.9   6.3   1.    0.  422.6   1. ]


## LightGBM

### single on ecoli

In [11]:
from utils.machine_learning.trainer import ml_trainer

In [12]:
ecoli_B_lgbm = LGBMClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_B_lgbm,'ecoli','B',cv,'average_precision',False)

**** Training on ecoli with type B ****

average_precision: 0.611 (0.065)
[[776  29]
 [ 23  28]]
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       805
         1.0       0.49      0.55      0.52        51

    accuracy                           0.94       856
   macro avg       0.73      0.76      0.74       856
weighted avg       0.94      0.94      0.94       856



In [13]:
ecoli_C_lgbm = LGBMClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_C_lgbm,'ecoli','C',cv,'roc_auc',False)

**** Training on ecoli with type C ****

roc_auc: 0.931 (0.022)
[[776  29]
 [ 24  27]]
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       805
         1.0       0.48      0.53      0.50        51

    accuracy                           0.94       856
   macro avg       0.73      0.75      0.74       856
weighted avg       0.94      0.94      0.94       856



In [16]:
ecoli_D_lgbm = LGBMClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_D_lgbm,'ecoli','D',cv,'roc_auc',False)

**** Training on ecoli with type D ****

roc_auc: 0.930 (0.020)
[[778  27]
 [ 26  25]]
              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97       805
         1.0       0.48      0.49      0.49        51

    accuracy                           0.94       856
   macro avg       0.72      0.73      0.73       856
weighted avg       0.94      0.94      0.94       856



In [17]:
ecoli_E_lgbm = LGBMClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_B_lgbm,'ecoli','E',cv,'roc_auc',False)

**** Training on ecoli with type E ****

roc_auc: 0.933 (0.020)
[[773  32]
 [ 22  29]]
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       805
         1.0       0.48      0.57      0.52        51

    accuracy                           0.94       856
   macro avg       0.72      0.76      0.74       856
weighted avg       0.94      0.94      0.94       856



### all species

In [18]:
all_B_lgbm = LGBMClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_B_lgbm,'all','B',cv,'roc_auc',False)

**** Training on all with type B ****

roc_auc: 0.943 (0.002)
[[35251   596]
 [ 2183  1008]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.63      0.32      0.42      3191

    accuracy                           0.93     39038
   macro avg       0.79      0.65      0.69     39038
weighted avg       0.92      0.93      0.92     39038



In [19]:
all_C_lgbm = LGBMClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_C_lgbm,'all','C',cv,'roc_auc',False)

**** Training on all with type C ****

roc_auc: 0.943 (0.002)
[[35248   599]
 [ 2191  1000]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.63      0.31      0.42      3191

    accuracy                           0.93     39038
   macro avg       0.78      0.65      0.69     39038
weighted avg       0.92      0.93      0.92     39038



In [20]:
all_D_lgbm = LGBMClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_D_lgbm,'all','D',cv,'roc_auc',False)

**** Training on all with type D ****

roc_auc: 0.933 (0.002)
[[35272   575]
 [ 2357   834]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.59      0.26      0.36      3191

    accuracy                           0.92     39038
   macro avg       0.76      0.62      0.66     39038
weighted avg       0.91      0.92      0.91     39038



In [21]:
all_E_lgbm = LGBMClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_E_lgbm,'all','E',cv,'roc_auc',False)

**** Training on all with type E ****

roc_auc: 0.939 (0.002)
[[35265   582]
 [ 2187  1004]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.63      0.31      0.42      3191

    accuracy                           0.93     39038
   macro avg       0.79      0.65      0.69     39038
weighted avg       0.92      0.93      0.92     39038



## HistGradientBoostingClassifier

### single on ecoli

In [22]:
# define the model
ecoli_B_HGBC = HistGradientBoostingClassifier(random_state=1)
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_B_HGBC,'ecoli','B',cv,'roc_auc',False)

**** Training on ecoli with type B ****

roc_auc: 0.929 (0.024)
[[776  29]
 [ 22  29]]
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       805
         1.0       0.50      0.57      0.53        51

    accuracy                           0.94       856
   macro avg       0.74      0.77      0.75       856
weighted avg       0.94      0.94      0.94       856



In [23]:
# define the model
ecoli_C_HGBC = HistGradientBoostingClassifier(random_state=1)
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_C_HGBC,'ecoli','C',cv,'roc_auc',False)

**** Training on ecoli with type C ****

roc_auc: 0.929 (0.023)
[[773  32]
 [ 23  28]]
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       805
         1.0       0.47      0.55      0.50        51

    accuracy                           0.94       856
   macro avg       0.72      0.75      0.74       856
weighted avg       0.94      0.94      0.94       856



In [24]:
# define the model
ecoli_D_HGBC = HistGradientBoostingClassifier(random_state=1)
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_D_HGBC,'ecoli','D',cv,'roc_auc',False)

**** Training on ecoli with type D ****

roc_auc: 0.930 (0.023)
[[773  32]
 [ 24  27]]
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       805
         1.0       0.46      0.53      0.49        51

    accuracy                           0.93       856
   macro avg       0.71      0.74      0.73       856
weighted avg       0.94      0.93      0.94       856



In [25]:
# define the model
ecoli_E_HGBC = HistGradientBoostingClassifier(random_state=1)
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_E_HGBC,'ecoli','E',cv,'roc_auc',False)

**** Training on ecoli with type E ****

roc_auc: 0.933 (0.021)
[[772  33]
 [ 19  32]]
              precision    recall  f1-score   support

         0.0       0.98      0.96      0.97       805
         1.0       0.49      0.63      0.55        51

    accuracy                           0.94       856
   macro avg       0.73      0.79      0.76       856
weighted avg       0.95      0.94      0.94       856



### all species

In [26]:
all_B_HGBC = HistGradientBoostingClassifier(random_state=1)
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_B_HGBC,'all','B',cv,'roc_auc',False)

**** Training on all with type B ****

roc_auc: 0.943 (0.002)
[[35237   610]
 [ 2182  1009]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.62      0.32      0.42      3191

    accuracy                           0.93     39038
   macro avg       0.78      0.65      0.69     39038
weighted avg       0.92      0.93      0.92     39038



In [27]:
all_C_HGBC = HistGradientBoostingClassifier(random_state=1)
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_C_HGBC,'all','C',cv,'roc_auc',False)

**** Training on all with type C ****

roc_auc: 0.943 (0.002)
[[35229   618]
 [ 2151  1040]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.63      0.33      0.43      3191

    accuracy                           0.93     39038
   macro avg       0.78      0.65      0.70     39038
weighted avg       0.92      0.93      0.92     39038



In [28]:
all_D_HGBC = HistGradientBoostingClassifier(random_state=1)
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_D_HGBC,'all','D',cv,'roc_auc',False)

**** Training on all with type D ****

roc_auc: 0.933 (0.002)
[[35287   560]
 [ 2367   824]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.60      0.26      0.36      3191

    accuracy                           0.93     39038
   macro avg       0.77      0.62      0.66     39038
weighted avg       0.91      0.93      0.91     39038



In [29]:
all_E_HGBC = HistGradientBoostingClassifier(random_state=1)
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_E_HGBC,'all','E',cv,'roc_auc',False)

**** Training on all with type E ****

roc_auc: 0.939 (0.002)
[[35239   608]
 [ 2178  1013]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.62      0.32      0.42      3191

    accuracy                           0.93     39038
   macro avg       0.78      0.65      0.69     39038
weighted avg       0.92      0.93      0.92     39038



## XGBoost

In [31]:
from xgboost import XGBClassifier

### single on ecoli

In [32]:
# define the model
ecoli_B_XGB = XGBClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_B_XGB,'ecoli','B',cv,'roc_auc',False)

**** Training on ecoli with type B ****

roc_auc: 0.925 (0.022)
[[774  31]
 [ 23  28]]
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       805
         1.0       0.47      0.55      0.51        51

    accuracy                           0.94       856
   macro avg       0.72      0.76      0.74       856
weighted avg       0.94      0.94      0.94       856



In [33]:
# define the model
ecoli_C_XGB = XGBClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_C_XGB,'ecoli','C',cv,'roc_auc',False)

**** Training on ecoli with type C ****

roc_auc: 0.926 (0.021)
[[775  30]
 [ 29  22]]
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96       805
         1.0       0.42      0.43      0.43        51

    accuracy                           0.93       856
   macro avg       0.69      0.70      0.70       856
weighted avg       0.93      0.93      0.93       856



In [34]:
# define the model
ecoli_D_XGB = XGBClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_D_XGB,'ecoli','D',cv,'roc_auc',False)

**** Training on ecoli with type D ****

roc_auc: 0.928 (0.021)
[[773  32]
 [ 24  27]]
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       805
         1.0       0.46      0.53      0.49        51

    accuracy                           0.93       856
   macro avg       0.71      0.74      0.73       856
weighted avg       0.94      0.93      0.94       856



In [35]:
# define the model
ecoli_E_XGB = XGBClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(ecoli_E_XGB,'ecoli','E',cv,'roc_auc',False)

**** Training on ecoli with type E ****

roc_auc: 0.929 (0.020)
[[775  30]
 [ 22  29]]
              precision    recall  f1-score   support

         0.0       0.97      0.96      0.97       805
         1.0       0.49      0.57      0.53        51

    accuracy                           0.94       856
   macro avg       0.73      0.77      0.75       856
weighted avg       0.94      0.94      0.94       856



### all species

In [36]:
# define the model
all_B_XGB = XGBClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_B_XGB,'all','B',cv,'roc_auc',False)

**** Training on all with type B ****

roc_auc: 0.944 (0.002)
[[35140   707]
 [ 2082  1109]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.61      0.35      0.44      3191

    accuracy                           0.93     39038
   macro avg       0.78      0.66      0.70     39038
weighted avg       0.92      0.93      0.92     39038



In [37]:
# define the model
all_C_XGB = XGBClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_C_XGB,'all','C',cv,'roc_auc',False)

**** Training on all with type C ****

roc_auc: 0.944 (0.002)
[[35161   686]
 [ 2066  1125]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.62      0.35      0.45      3191

    accuracy                           0.93     39038
   macro avg       0.78      0.67      0.71     39038
weighted avg       0.92      0.93      0.92     39038



In [38]:
# define the model
all_D_XGB = XGBClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_D_XGB,'all','D',cv,'roc_auc',False)

**** Training on all with type D ****

roc_auc: 0.932 (0.002)
[[35246   601]
 [ 2339   852]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.59      0.27      0.37      3191

    accuracy                           0.92     39038
   macro avg       0.76      0.63      0.66     39038
weighted avg       0.91      0.92      0.91     39038



In [39]:
# define the model
all_E_XGB = XGBClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
ml_trainer(all_E_XGB,'all','E',cv,'roc_auc',False)

**** Training on all with type E ****

roc_auc: 0.939 (0.002)
[[35214   633]
 [ 2163  1028]]
              precision    recall  f1-score   support

         0.0       0.94      0.98      0.96     35847
         1.0       0.62      0.32      0.42      3191

    accuracy                           0.93     39038
   macro avg       0.78      0.65      0.69     39038
weighted avg       0.92      0.93      0.92     39038



# DL

In [6]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


## Dataset

### Single genome

In [1]:
from utils.deep_learning.data_loader import SingleSpecies

In [4]:
test_single = SingleSpecies(species="bsub",data_type="B")

test_single[500]

{'gc_signature': array([ 0.000e+00,  1.173e+03,  3.980e+01, -2.200e+00,  2.450e+01,
        -2.200e+00,  2.810e+01, -2.100e+00,  3.080e+01, -3.000e+00,
         1.490e+01,  1.000e+00]),
 'hgt': 1.0}

### All Genome

In [25]:
from utils.deep_learning.data_loader import AllSpecies

In [26]:
test_all = AllSpecies(data_type="B")

test_all[500]

True
           Strand  Length   GC1  SD1   GC2  SD2   GC3  SD3   GCT  SDT    Mah   
ID                                                                             
nana_3260     NaN       0  60.4  0.9  31.6 -1.5  44.3 -0.4  45.4 -0.4   19.6  \
nanb_3261     NaN       0  58.5  0.6  40.1  0.6  45.5 -0.2  48.0  0.3    9.3   
nanb_1695     NaN       0  53.0 -0.4  34.5 -0.9  54.2  0.6  47.2 -0.2   34.2   
nana_1696     NaN       0  34.1 -4.6  29.3 -2.3  31.7 -2.9  31.7 -4.8  326.1   
SCO3176       1.0    1749   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN    NaN   

           AADev  HGT  
ID                     
nana_3260      1    0  
nanb_3261      1    0  
nanb_1695      1    0  
nana_1696      1    0  
SCO3176        0    0  


{'gc_signature': tensor([0.0000e+00, 4.0200e+02, 3.9300e+01, 3.0000e-01, 4.5900e+01, 3.5000e+00,
         2.0700e+01, 0.0000e+00, 3.5300e+01, 1.9000e+00, 1.6320e+02, 1.0000e+00],
        dtype=torch.float64),
 'hgt': tensor(0., dtype=torch.float64)}

## Models

### Binary Classifier MLP

https://medium.com/biaslyai/pytorch-introduction-to-neural-network-feedforward-neural-network-model-e7231cff47cb

In [27]:
class Feedforward(torch.nn.Module):
    def __init__(self, input_size):
        super(Feedforward, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 100).double()
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(100, 1).double()
        self.sigmoid1 = torch.nn.Sigmoid()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.sigmoid1(x)
        return x

### MTL

In [28]:
# for MTL need to change 

class MTLFeedforward(torch.nn.Module):
    def __init__(self, input_size):
        super(Feedforward, self).__init__()
        self.input_size = input_size
        self.fc1 = torch.nn.Linear(self.input_size, 100).double()
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(100, 50).double()
        
        
        # Layers for species classification
        self.species_relu1 = torch.nn.ReLU()
        self.species_fc1 = torch.nn.Linear(50, 1).double()
        self.species_sigmoid1 = torch.nn.Sigmoid()
        
        # Layers for HGT classifications
        self.hgt_relu1 = torch.nn.ReLU()
        self.hgt_fc1 = torch.nn.Linear(50, 1).double()
        self.hgt_sigmoid1 = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        
        # fw pass to species layer
        x_species = self.species_relu1(x)
        x_species = self.species_fc1(x_species)
        x_species = self.species_sigmoid1(x_species)
        
        # fw pass to species layer
        x_hgt = self.hgt_relu1(x)
        x_hgt = self.hgt_fc1(x_hgt)
        x_hgt = self.hgt_sigmoid1(x_hgt)
        
        
        return x_species, x_hgt

## Training

### Helper function: trainer class

In [29]:
from utils.deep_learning.trainer import Classifier

### Binary Classifier Trained on single species

#### init data

In [9]:
# create dataset for single species
ecoli_B_set= SingleSpecies(species="ecoli",data_type="B")
ecoli_C_set= SingleSpecies(species="ecoli",data_type="C")
ecoli_D_set= SingleSpecies(species="ecoli",data_type="D")
ecoli_E_set= SingleSpecies(species="ecoli",data_type="E")

# partition
# Partitioning like this is not good. 
# Prolly do a csv partitioning again so it is somewhat more reproducible
ecoli_B_train_set, ecoli_B_test_set = torch.utils.data.random_split(ecoli_B_set, [0.7, 0.3])
ecoli_C_train_set, ecoli_C_test_set = torch.utils.data.random_split(ecoli_C_set, [0.7, 0.3])
ecoli_D_train_set, ecoli_D_test_set = torch.utils.data.random_split(ecoli_D_set, [0.7, 0.3])
ecoli_E_train_set, ecoli_E_test_set = torch.utils.data.random_split(ecoli_E_set, [0.7, 0.3])

In [10]:
print(ecoli_B_train_set[0])
print(ecoli_C_train_set[0])
print(ecoli_D_train_set[0])
print(ecoli_E_train_set[0])

{'gc_signature': array([ 0.00e+00,  5.46e+02,  6.01e+01,  3.00e-01,  3.99e+01, -1.00e-01,
        5.79e+01,  4.00e-01,  5.26e+01,  3.00e-01,  2.46e+01,  1.00e+00]), 'hgt': 0.0}
{'gc_signature': array([1101. ,   45.7,   -2.1,   27.7,   -2.9,   33.4,   -2.6,   35.6,
         -3.2,   16.3]), 'hgt': 1.0}
{'gc_signature': array([-1.1,  1.4, -0.5, -0.3]), 'hgt': 0.0}
{'gc_signature': array([ 0.000e+00,  1.002e+03,  1.200e+00,  1.000e+00, -5.000e-01,
        6.000e-01,  1.290e+01,  1.000e+00]), 'hgt': 0.0}


#### train models

In [32]:
ecoli_B_classifier = Classifier(Feedforward(12),torch.nn.BCELoss(), torch.optim.Adam,0.00001)
ecoli_B_classifier.train(ecoli_B_train_set, 5,20)

EPOCH 1:
LOSS train 3.4214953573115516
EPOCH 2:
LOSS train 1.2520247603435526
EPOCH 3:
LOSS train 0.5771831622110641
EPOCH 4:
LOSS train 0.39535662853452874
EPOCH 5:
LOSS train 0.3494302204837994
EPOCH 6:
LOSS train 0.33462268718869903
EPOCH 7:
LOSS train 0.32725941945975695
EPOCH 8:
LOSS train 0.32203289615911695
EPOCH 9:
LOSS train 0.32018248829778667
EPOCH 10:
LOSS train 0.31692508641399236
EPOCH 11:
LOSS train 0.31380003853452443
EPOCH 12:
LOSS train 0.3111193341865896
EPOCH 13:
LOSS train 0.31076782787326857
EPOCH 14:
LOSS train 0.30827637951569753
EPOCH 15:
LOSS train 0.3080431990042782
EPOCH 16:
LOSS train 0.3053316019669697
EPOCH 17:
LOSS train 0.3041932128238745
EPOCH 18:
LOSS train 0.30331871846449177
EPOCH 19:
LOSS train 0.30311918960163897
EPOCH 20:
LOSS train 0.3026257736239527


In [36]:
ecoli_C_classifier = Classifier(Feedforward(10),torch.nn.BCELoss(), torch.optim.Adam,0.00001)
ecoli_C_classifier.train(ecoli_C_train_set, 5,20)

EPOCH 1:
LOSS train 2.1744859579745106
EPOCH 2:
LOSS train 0.38089901222470646
EPOCH 3:
LOSS train 0.3348017417785868
EPOCH 4:
LOSS train 0.32245541985765847
EPOCH 5:
LOSS train 0.31357476294154296
EPOCH 6:
LOSS train 0.3097535535441396
EPOCH 7:
LOSS train 0.3045742665482406
EPOCH 8:
LOSS train 0.303356848816035
EPOCH 9:
LOSS train 0.2983815035390075
EPOCH 10:
LOSS train 0.2974454736823319
EPOCH 11:
LOSS train 0.2965568908188139
EPOCH 12:
LOSS train 0.29554000882101766
EPOCH 13:
LOSS train 0.3003131101211793
EPOCH 14:
LOSS train 0.29422725215689927
EPOCH 15:
LOSS train 0.292108851971414
EPOCH 16:
LOSS train 0.29295821447960535
EPOCH 17:
LOSS train 0.29330132094694517
EPOCH 18:
LOSS train 0.2947391128216287
EPOCH 19:
LOSS train 0.29172456795317653
EPOCH 20:
LOSS train 0.29097027047698554


In [37]:
ecoli_D_classifier = Classifier(Feedforward(4),torch.nn.BCELoss(), torch.optim.Adam,0.001)
ecoli_D_classifier.train(ecoli_D_train_set, 5,20)

EPOCH 1:
LOSS train 0.31665559461123866
EPOCH 2:
LOSS train 0.24832329665270944
EPOCH 3:
LOSS train 0.23494483671892752
EPOCH 4:
LOSS train 0.22791907392675154
EPOCH 5:
LOSS train 0.22302981641214556
EPOCH 6:
LOSS train 0.2202641857338765
EPOCH 7:
LOSS train 0.21840084022822828
EPOCH 8:
LOSS train 0.2160855691195408
EPOCH 9:
LOSS train 0.2151486137874429
EPOCH 10:
LOSS train 0.21373747222992773
EPOCH 11:
LOSS train 0.21116681984443056
EPOCH 12:
LOSS train 0.21064996643279932
EPOCH 13:
LOSS train 0.21186448055864038
EPOCH 14:
LOSS train 0.2108982067608469
EPOCH 15:
LOSS train 0.21124463475440328
EPOCH 16:
LOSS train 0.2109814915363215
EPOCH 17:
LOSS train 0.20898984060329887
EPOCH 18:
LOSS train 0.2091291941214206
EPOCH 19:
LOSS train 0.2091217259078254
EPOCH 20:
LOSS train 0.21042171552471453


In [38]:
ecoli_E_classifier = Classifier(Feedforward(8),torch.nn.BCELoss(), torch.optim.Adam,0.001)
ecoli_E_classifier.train(ecoli_E_train_set, 5,20)

EPOCH 1:
LOSS train 0.906980384816023
EPOCH 2:
LOSS train 0.6924047927882065
EPOCH 3:
LOSS train 0.6183962320582325
EPOCH 4:
LOSS train 0.5654943359405175
EPOCH 5:
LOSS train 0.6857722903103958
EPOCH 6:
LOSS train 0.7190535973257559
EPOCH 7:
LOSS train 0.49188293220255924
EPOCH 8:
LOSS train 0.5905085982096737
EPOCH 9:
LOSS train 0.5111248401961138
EPOCH 10:
LOSS train 0.5592117121783459
EPOCH 11:
LOSS train 0.4703963749841636
EPOCH 12:
LOSS train 0.5040785971270321
EPOCH 13:
LOSS train 0.574630162195057
EPOCH 14:
LOSS train 0.5545703778592008
EPOCH 15:
LOSS train 0.41598716612407566
EPOCH 16:
LOSS train 0.4700934209807114
EPOCH 17:
LOSS train 0.450022495238464
EPOCH 18:
LOSS train 0.45073833769926147
EPOCH 19:
LOSS train 0.4305160684827943
EPOCH 20:
LOSS train 0.41927602059447633


In [40]:
ecoli_B_classifier.eval(ecoli_B_test_set)

              precision    recall  f1-score   support

     Non-HGT       0.89      1.00      0.94      1412
         HGT       0.73      0.08      0.15       196

    accuracy                           0.88      1608
   macro avg       0.81      0.54      0.54      1608
weighted avg       0.87      0.88      0.84      1608



In [41]:
ecoli_C_classifier.eval(ecoli_C_test_set)

              precision    recall  f1-score   support

     Non-HGT       0.89      0.99      0.94      1420
         HGT       0.68      0.09      0.16       188

    accuracy                           0.89      1608
   macro avg       0.79      0.54      0.55      1608
weighted avg       0.87      0.89      0.85      1608



In [42]:
ecoli_D_classifier.eval(ecoli_D_test_set)

              precision    recall  f1-score   support

     Non-HGT       0.92      0.97      0.95      1425
         HGT       0.61      0.35      0.44       183

    accuracy                           0.90      1608
   macro avg       0.77      0.66      0.69      1608
weighted avg       0.89      0.90      0.89      1608



In [43]:
ecoli_E_classifier.eval(ecoli_E_test_set)

              precision    recall  f1-score   support

     Non-HGT       0.90      0.99      0.94      1415
         HGT       0.62      0.17      0.27       193

    accuracy                           0.89      1608
   macro avg       0.76      0.58      0.60      1608
weighted avg       0.86      0.89      0.86      1608



### Binary Classifer on All

In [99]:
allspecies_B = AllSpecies(data_type="B")

True
           Strand  Length   GC1  SD1   GC2  SD2   GC3  SD3   GCT  SDT    Mah   
ID                                                                             
nana_3260     NaN       0  60.4  0.9  31.6 -1.5  44.3 -0.4  45.4 -0.4   19.6  \
nanb_3261     NaN       0  58.5  0.6  40.1  0.6  45.5 -0.2  48.0  0.3    9.3   
nanb_1695     NaN       0  53.0 -0.4  34.5 -0.9  54.2  0.6  47.2 -0.2   34.2   
nana_1696     NaN       0  34.1 -4.6  29.3 -2.3  31.7 -2.9  31.7 -4.8  326.1   
SCO3176       1.0    1749   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN    NaN   

           AADev  HGT  
ID                     
nana_3260      1    0  
nanb_3261      1    0  
nanb_1695      1    0  
nana_1696      1    0  
SCO3176        0    0  


In [100]:
allspecies_C = AllSpecies(data_type="C",normalize=True)

True
         Length  GC1  SD1  GC2  SD2  GC3  SD3  GCT  SDT  Mah  HGT
ID                                                               
SCO3176    1749  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN    0


In [101]:
allspecies_D = AllSpecies(data_type="D",normalize=True)

True
         SD1  SD2  SD3  SDT  HGT
ID                              
SCO3176  NaN  NaN  NaN  NaN    0


In [115]:

# Partition data
allspecies_B_train_set, allspecies_B_test_set = torch.utils.data.random_split(allspecies_B, [0.7, 0.3])
allspecies_C_train_set, allspecies_C_test_set = torch.utils.data.random_split(allspecies_C, [0.7, 0.3])
allspecies_D_train_set, allspecies_D_test_set = torch.utils.data.random_split(allspecies_D, [0.7, 0.3])


In [116]:
print(allspecies_B_train_set[0])
print(allspecies_C_train_set[0])
print(allspecies_D_train_set[0])

{'gc_signature': tensor([ 0.0000e+00,  1.2360e+03,  5.6700e+01, -7.0000e-01,  4.7000e+01,
         6.0000e-01,  4.6500e+01, -1.3000e+00,  5.0000e+01, -7.0000e-01,
         3.3100e+01,  1.0000e+00], dtype=torch.float64), 'hgt': tensor(0., dtype=torch.float64)}
{'gc_signature': tensor([0.0299, 0.4344, 0.4513, 0.2773, 0.3650, 0.2064, 0.4612, 0.2828, 0.6066,
        0.0061], dtype=torch.float64), 'hgt': tensor(0., dtype=torch.float64)}
{'gc_signature': tensor([0.5436, 0.4100, 0.4201, 0.6667], dtype=torch.float64), 'hgt': tensor(0., dtype=torch.float64)}


In [137]:
bacteria_classifier_B = Classifier(Feedforward(12), torch.nn.BCELoss(), torch.optim.Adam, 0.001)
bacteria_classifier_C = Classifier(Feedforward(10), torch.nn.BCELoss(), torch.optim.Adam, 0.001)
bacteria_classifier_D = Classifier(Feedforward(4), torch.nn.BCELoss(), torch.optim.Adam, 0.001)

In [138]:
bacteria_classifier_B.train(allspecies_B_train_set,2,10)

EPOCH 1:
LOSS train 0.3246158487945819
EPOCH 2:


KeyboardInterrupt: 

In [139]:
bacteria_classifier_C.train(allspecies_C_train_set,1,10)

EPOCH 1:
LOSS train 0.223483000962128
EPOCH 2:


KeyboardInterrupt: 

In [ ]:
bacteria_classifier_D.train(allspecies_D_train_set,5,10)

EPOCH 1:


ValueError: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.

# Test

In [4]:
import os
def create_csv_list(preprocessed_path):
    list_csv_files = []
    for path in os.listdir(preprocessed_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(preprocessed_path, path)):
            list_csv_files.append(os.path.join(preprocessed_path, path))
            
    return list_csv_files

In [5]:
lists = create_csv_list("../EDA/preprocessed_data")

In [6]:
print(lists)

['../EDA/preprocessed_data/bbur.csv', '../EDA/preprocessed_data/bmelic2.csv', '../EDA/preprocessed_data/llac.csv', '../EDA/preprocessed_data/sau2.csv', '../EDA/preprocessed_data/cjen.csv', '../EDA/preprocessed_data/bsub.csv', '../EDA/preprocessed_data/sent.csv', '../EDA/preprocessed_data/vvul1c1.csv', '../EDA/preprocessed_data/nsp.csv', '../EDA/preprocessed_data/atum1c2.csv', '../EDA/preprocessed_data/bhal.csv', '../EDA/preprocessed_data/cperf.csv', '../EDA/preprocessed_data/mtub2.csv', '../EDA/preprocessed_data/bsp.csv', '../EDA/preprocessed_data/nmen1.csv', '../EDA/preprocessed_data/ecoli2.csv', '../EDA/preprocessed_data/pmul.csv', '../EDA/preprocessed_data/spneu1.csv', '../EDA/preprocessed_data/atum2c2.csv', '../EDA/preprocessed_data/sau3.csv', '../EDA/preprocessed_data/mpul.csv', '../EDA/preprocessed_data/fnucl.csv', '../EDA/preprocessed_data/dra1.csv', '../EDA/preprocessed_data/ecoli3.csv', '../EDA/preprocessed_data/ctep.csv', '../EDA/preprocessed_data/cpneu.csv', '../EDA/preproce

In [7]:
print(len(lists))

78
